## Using physics informed neural networks (PINNs) to solve a 2-D head equation

In the following notebook, we implement as physics informed neural network to solve heat equation in two-dimenson. The program implements the following parabolic equation

$$\frac{\partial u(\boldsymbol{x},t)}{\partial t} - \Delta u(\boldsymbol{x},t) = f(\boldsymbol{x}, t) \ \forall \boldsymbol{x} \in \Omega,\ t \in (0, T)$$ 
$$u(\boldsymbol{x,0}) = u_{0}(\boldsymbol{x})\ \forall \boldsymbol{x} \in \Omega$$
$$u(\boldsymbol{x,t}) = g(\boldsymbol{x})\ \forall \boldsymbol{x} \in \partial\Omega,\ t\in (0,T)$$

$\Omega$ is the domain in $\mathbb{R}^d$, $\partial \Omega$ is the boundary of the domain, $T$ is the final time, $u_0: \Omega \rightarrow \mathbb{R}^d$ is the initial condition

### Methodology

The approach is to construct a neural network approximation $u_{\theta}(\boldsymbol{x}, t) \approx u(\boldsymbol{x}, t)$ of the solution of the PDE. Here $u_{\theta}: \Omega \times[0,T] \rightarrow \mathbb{R}^d$. 

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
# Set data type
DTYPE = 'float32'
tf.keras.backend.set_floatx(DTYPE)

In [5]:
# Set the value of constants
pi = tf.constant(np.pi, dtype=DTYPE)

2022-11-05 16:03:32.344176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Set the PDE

# Set the initial condition
def fun_u_0(x):
    n = x.shape[0]
    return tf.zeros((n,1), dtype=DTYPE)

In [7]:
# Set the boundary condition
def fun_u_b(x, t):
    n = x.shape[0]
    return tf.zeros((n,1), dtype=DTYPE)

In [8]:
# Set the right hand side
import math
period = 0.2
def heat_src(x,t):
    n = x.shape[0]
    point_within_period = t/period - math.cel(t/period)
    if(point_within_period >= 0.0 and point_within_period <= 0.2):
        if((x[0] > 0.5) and (x[1] > -0.5)):
            return tf.ones((n,1), dtype=DTYPE)
        else:
            return tf.zeros((n,1), dtype=DTYPE)
    elif(point_within_period >= 0.5 and point_within_period <= 0.7):
        if((x[0] > -0.5) and (x[1] > 0.5)):
            return tf.ones((n,1), dtype=DTYPE)
        else:
            return tf.zeros((n,1), dtype=DTYPE)
    else:
        return tf.zeros((n,1), dtype=DTYPE)

In [9]:
# Define the residual
def fun_residual(x, t, f_x, u_xx, u_yy, u_t, u):
    return f_x + u_xx + u_yy - u_t

In [10]:
# Set Final Time
T = tf.constant(1., dtype=DTYPE)

# Spatial dimensions
dim = 2

In [11]:
a = np.zeros((dim), dtype = DTYPE)
b = np.zeros((dim), dtype = DTYPE)

In [12]:
# Set the number of data points

N_0 = 100
N_b = 100
N_r = 10000

# Set boundary
tmin = 0.0
tmax = 1.0
xmin = -1.0
xmax = 1.0

# Lower bounds
lb = tf.constant([tmin, xmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, xmax], dtype=DTYPE)

# Random seed for reproducible results
tf.random.set_seed(42)


In [ ]:
@tf.function
def draw_X(num_samples, a, b):
    dim = a.shape[0]
    
